### Project 1

We need to design an dimplement a class that will be used to represent bank accounts.

We want the following functionality and characteristics:
- accounts are uniquely identified by an **account number** (assume it will just be passed in the initializer)
- account holders have a **first** and **last** name
- accounts have an associated **preferred time zone offset** (e.g. -7 for MST)
- **balances** need to be zero or higher, and should not be directly settable.
- but, **deposits** and **withdrawals** can be made (given sufficient funds)
    - if a withdrawal is attempted that would result in nagative funds, the transaction should be declined.
- a **monthly interest rate** exists and is applicable to all accounts **uniformly**. There should be a method that can be called to calculate the interest on the current balance using the current interest rate, and **add it** to the balance.
- each deposit and withdrawal must generate a **confirmation number** composed of:
    - the transaction type: `D` for deposit, and `W` for withdrawal, `I` for interest deposit, and `X` for declined (in which case the balance remains unaffected)
    - the account number
    - the time the transaction was made, using UTC
    - an incrementing number (that increments across all accounts and transactions)
    - for (extreme!) simplicity assume that the transaction id starts at zero (or whatever number you choose) whenever the program starts
    - the confirmation number should be returned from any of the transaction methods (deposit, withdraw, etc)
- create a **method** that, given a confirmation number, returns:
    - the account number, transaction code (D, W, etc), datetime (UTC format), date time (in whatever timezone is specified in te argument, but more human readable), the transaction ID
    - make it so it is a nicely structured object (so can use dotted notation to access these three attributes)
    - I purposefully made it so the desired timezone is passed as an argument. Can you figure out why? (hint: does this method require any information from any instance?)

For example, we may have an account with:
- account number `140568` 
- preferred time zone offset of -7 (MST) 
- an existing balance of `100.00`

Suppose the last transaction ID in the system was `123`, and a deposit is made for `50.00` on `2019-03-15T14:59:00` (UTC) on that account (or `2019-03-15T07:59:00` in account's preferred time zone offset)

The new balance should reflect `150.00` and the confirmation number returned should look something like this:

```D-140568-20190315145900-124```

We also want a method that given the confirmation number returns an object with attributes:
- `result.account_number` --> `140568`
- `result.transaction_code` --> `D`
- `result.transaction_id` --> `124`
- `result.time` --> `2019-03-15 07:59:00 (MST)`
- `result.time_utc` --> `2019-03-15T14:59:00`

Furthermore, if current interest rate is `0.5%`, and the account's balance is `1000.00`, then the result of calling the `deposit_interest` (or whatever name you choose) method, should result in a new transaction and a new balance of `1050.00`. Calling this method should also return a confirmation number.

For simplicty, just use floats, but be aware that for these types of situations you'll probably want to use `Decimal` objects instead of floats.

There are going to be many ways to design something like this, especially since I have not nailed down all the specific requirements, so you'll have to fill the gaps yourself and decide what other things you may want to implement (like is the account number going to be a mutable property, or "read-only" and so on).

See how many different ideas you can use from what we covered in the last section. 

My approach will end up creating two classes: a `TimeZone` class used to store the time zone name and offset definition (in hours and minutes), and a main class called `Account` that will have the following "public" interface:
- initializer with account number, first name, last name, optional preferred time zone, starting balance (defaults to 0)
- a first name property (read/write)
- a last name property (read/write)
- a full name property (computed, read-only)
- a balance property (read-only)
- an interest rate property (class level property)
- deposit, withdraw, pay_interest methods
- parse confirmation code

Class will have additional state and methods, but those will be used for implementation.

You should also remember to test your code! In the solutions I will introduce you to Python's `unittest` package. Even if you skip this project, at least review that video and/or notebook if you are unfamiliar with `unittest`.

In [1]:
from collections import namedtuple
from datetime import datetime, timedelta
import itertools
import numbers

In [2]:
class TimeZone:
    def __init__(self, name, offset_hours, offset_minutes):
        if name is None or len(str(name).strip()) == 0:
            raise ValueError("Timezone name cannot be empty")
        self._name = str(name).strip()
        
        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError(f"Hour offset must be an integer, got {type(offset_hours).__name__} type.")
        
        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError(f"Minutes offset must be an integer, got {type(offset_minutes).__name__} type.")
        
        if abs(offset_minutes) > 59:
            raise ValueError("Minutes offset must be between -59 and 59 inclusive.")
        
        total_offset = timedelta(hours=offset_hours, minutes=offset_minutes)
        if not timedelta(hours=-12) <= total_offset <= timedelta(hours=14):
            raise ValueError("Offset must be between -12 and 14.")
        
        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._total_offset = total_offset
        
    @property
    def total_offset(self):
        return self._total_offset
    
    @property
    def name(self):
        return self._name
    
    def __eq__(self, other):
        return (isinstance(other, TimeZone) and 
                self._name == other._name and
                self._offset_hours == other._offset_hours and
                self._offset_minutes == other._offset_minutes)
    
    def __repr__(self):
        return (f"Timezone(name='{self._name}', "
                f"offset_hours={self._offset_hours}, "
                f"offset_minutes={self._offset_minutes})")

In [3]:
Confirmation = namedtuple("Confirmation", "account_number transaction_code transaction_id time_utc time")

In [46]:
class Account:
    transaction_counter = itertools.count(100)
    _interest_rate = 0.5
    
    _transaction_codes = {
        "deposit": "D",
        "withdraw": "W",
        "interest": "I",
        "rejected": "X"
    }
    
    def __init__(self, account_number, first_name, last_name,
                 timezone=None, initial_balance=0):
        self._account_number = account_number
        self.first_name = first_name
        self.last_name = last_name
        
        if timezone is None:
            timezone = TimeZone("UTC", 0, 0)
        self.timezone = timezone
        
        self._balance = Account.validate_real_number(initial_balance, min_value=0)
        
    @property
    def account_number(self):
        return self._account_number
    
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        self.validate_and_set_name("_first_name", value, "First name")

    @property
    def last_name(self):
        return self._last_name
    
    @property
    def balance(self):
        return self._balance
    
    @last_name.setter
    def last_name(self, value):
        self.validate_and_set_name("_last_name", value, "Last name")
        
    @property
    def timezone(self):
        return self._timezone
    
    @timezone.setter
    def timezone(self, value):
        if not isinstance(value, TimeZone):
            raise ValueError(f"Timezone must be a valid TimeZone object, got {type(value).__name__} type.")
        self._timezone = value
        
    @classmethod
    def get_interest_rate(cls):
        return cls._interest_rate
    
    @classmethod
    def set_interest_rate(cls, value):
        if not isinstance(value, numbers.Real):
            raise ValueError(f"Interest rate must be a real number, got {type(value).__name__} type.")
        if value < 0:
            raise ValueError("Interest rate cannot be negative.")
        cls._interest_rate = value
        
    @staticmethod
    def validate_real_number(value, min_value=None):
        if not isinstance(value, numbers.Real):
            raise ValueError(f"Value must be a real number, got {type(value).__name__} type.")
        if min_value is not None and value < min_value:
            raise ValueError(f"Value must be at least {min_value}.")
        return value
     
    def generate_confirmation_code(self, transaction_code):
        dt_str = datetime.now().strftime("%Y%m%d%H%M%S")
        return f"{transaction_code}-{self.account_number}-{dt_str}-{next(Account.transaction_counter)}"
    
    def validate_and_set_name(self, attr_name, value, field_title):
        if value is None or len(str(value).strip()) == 0:
            raise ValueError(f"{field_title} cannot be empty.")
        setattr(self, attr_name, str(value).strip())
        
    def make_transaction(self):
        return self.generate_confirmation_code("dummy")
    
    @staticmethod
    def parse_confirmation_code(confirmation_code, preferred_time_zone = None):
        parts = confirmation_code.split("-")
        if len(parts) != 4:
            raise ValueError(f"Invalid confirmation code.")
        
        transaction_code, account_number, raw_dt_utc, transaction_id = parts
        
        try:
            dt_utc = datetime.strptime(raw_dt_utc, "%Y%m%d%H%M%S")
        except ValueError as e:
            raise ValueError(f"Invalid transaction datetime.") from e
        
        if preferred_time_zone is None:
            preferred_time_zone = TimeZone("UTC", 0, 0)
            
        if not isinstance(preferred_time_zone, TimeZone):
            raise ValueError(f"Invalid TimeZone specified, got {type(preferred_time_zone).__name__} type.")

        dt_preferred = dt_utc + preferred_time_zone.total_offset
        dt_preferred_str = f"{dt_preferred.strftime('%Y-%m-%d %H:%M:%S')} ({preferred_time_zone.name})"
        
        return Confirmation(account_number, transaction_code, transaction_id, dt_utc.isoformat(), dt_preferred_str)
    
    def deposit(self, value):
        value = Account.validate_real_number(value, 0.01)
        
        transaction_code = Account._transaction_codes["deposit"]
        
        conf_code = self.generate_confirmation_code(transaction_code)
        self._balance += value
        return conf_code
    
    def withdraw(self, value):
        value = Account.validate_real_number(value, 0.01)

        accepted = False
        if self._balance - value < 0:
            transaction_code = Account._transaction_codes["rejected"]
        else:
            accepted = True
            transaction_code = Account._transaction_codes["withdraw"]
        
        conf_code = self.generate_confirmation_code(transaction_code)
        if accepted:
            self._balance -= value

        return conf_code
    
    def pay_interest(self):
        interest = self.balance * Account.get_interest_rate() / 100
        conf_code = self.generate_confirmation_code(Account._transaction_codes["interest"])
        self._balance += interest
        return conf_code

In [13]:
a = Account("A100", "Eric", "Idle", timezone=TimeZone("MST", -7, 0), initial_balance=100)

In [14]:
print(a.balance)
print(a.deposit(150.02))
print(a.balance)
print(a.withdraw(0.02))
print(a.balance)
Account.set_interest_rate(1.0)
print(a.get_interest_rate())
print(a.pay_interest())
print(a.balance)
print(a.withdraw(1000))

100.0
D-A100-20240513211504-104
250.02
W-A100-20240513211504-105
250.0
1.0
I-A100-20240513211504-106
252.5
X-A100-20240513211504-107


In [15]:
import unittest

In [19]:
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

In [28]:
class TestAccount(unittest.TestCase):
    def setUp(self):
        print("Running setup...")
        self.x = 100
    
    def tearDown(self):
        print("Running teardown...")
    
    def test_ok(self):
        self.x = 200
        self.assertEqual(200, self.x)
        
    def test_2(self):
        self.assertEqual(140, self.x)

In [29]:
run_tests(TestAccount)

test_2 (__main__.TestAccount.test_2) ... FAIL
test_ok (__main__.TestAccount.test_ok) ... ok

FAIL: test_2 (__main__.TestAccount.test_2)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_13720/4199318728.py", line 14, in test_2
    self.assertEqual(140, self.x)
AssertionError: 140 != 100

----------------------------------------------------------------------
Ran 2 tests in 0.002s

FAILED (failures=1)


Running setup...
Running teardown...
Running setup...
Running teardown...


In [57]:
class TestAccount(unittest.TestCase):
    
    def setUp(self):
        self.account_number = "A100"
        self.first_name = "FIRST"
        self.last_name = "LAST"
        self.tz = TimeZone("TZ", 1, 30)
        self.balance = 100.00
        
    def create_account(self):
        return Account(self.account_number, self.first_name, self.last_name, self.tz, self.balance)
    
    def test_create_timezone(self):
        tz = TimeZone("ABC", -1, -30)
        self.assertEqual("ABC", tz.name)
        self.assertEqual(timedelta(hours=-1, minutes=-30), tz.total_offset)
        
    def test_timezones_equal(self):
        tz1 = TimeZone("ABC", -1, -30)
        tz2 = TimeZone("ABC", -1, -30)
        self.assertEqual(tz1, tz2)
        
    def test_timezones_not_equal(self):
        tz = TimeZone("ABC", -1, -30)
        test_timezones = (
            TimeZone("DEF", -1, -30),
            TimeZone("ABC", -1, 0),
            TimeZone("ABC", 10, -30),
        )
        
        for i, test_tz in enumerate(test_timezones):
            with self.subTest(test_number=f"Test # {i}"):
                self.assertNotEqual(test_tz, tz)
                
    def test_create_account(self):
        a = self.create_account()
        
        self.assertEqual(self.account_number, a.account_number)
        self.assertEqual(self.first_name, a.first_name)
        self.assertEqual(self.last_name, a.last_name)
        self.assertEqual(self.tz, a.timezone)
        self.assertEqual(self.balance, a.balance)
        
    def test_create_blank_first_name(self):
        self.first_name = ""
        
        with self.assertRaises(ValueError):
            a = self.create_account()
            
    def test_create_account_negative_balance(self):
        self.balance = -1000
        
        with self.assertRaises(ValueError):
            a = self.create_account()
            
    def test_account_withdraw_ok(self):
        withdraw_amount = 20
        a = self.create_account()
        conf_code = a.withdraw(withdraw_amount)
        self.assertTrue(conf_code.startswith("W-"))
        self.assertEqual(self.balance - withdraw_amount, a.balance)
        
        
    def test_account_withdraw_overdraw(self):
        withdraw_amount = 200
        a = self.create_account()
        conf_code = a.withdraw(withdraw_amount)
        self.assertTrue(conf_code.startswith("X-"))
        self.assertEqual(self.balance, a.balance)

In [58]:
run_tests(TestAccount)

test_account_withdraw_ok (__main__.TestAccount.test_account_withdraw_ok) ... ok
test_account_withdraw_overdraw (__main__.TestAccount.test_account_withdraw_overdraw) ... ok
test_create_account (__main__.TestAccount.test_create_account) ... ok
test_create_account_negative_balance (__main__.TestAccount.test_create_account_negative_balance) ... ok
test_create_blank_first_name (__main__.TestAccount.test_create_blank_first_name) ... ok
test_create_timezone (__main__.TestAccount.test_create_timezone) ... ok
test_timezones_equal (__main__.TestAccount.test_timezones_equal) ... ok
test_timezones_not_equal (__main__.TestAccount.test_timezones_not_equal) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.007s

OK
